# D365FO MCP LangGraph Agent

This notebook demonstrates how to create a LangGraph agent that can interact with Microsoft Dynamics 365 Finance & Operations (D365FO) using the Model Context Protocol (MCP). The agent can query D365FO data entities, download documents, and perform various operations on the ERP system.

## Overview

The notebook covers:
- Setting up API keys for different LLM providers (OpenAI, Google AI, Azure OpenAI)
- Connecting to D365FO via MCP client
- Creating custom tools for document download
- Building an intelligent agent that can interact with D365FO
- Testing various D365FO operations through the agent

## Prerequisites

Before running this notebook, ensure you have:
- Access to a D365FO environment
- D365FO service principal credentials (Client ID, Client Secret, Tenant ID)
- API keys for your preferred LLM provider
- Docker installed (for running the D365FO MCP server)

## 1. Import Required Libraries

First, we import the necessary libraries for the LangGraph agent and LLM initialization.

In [1]:
import os
from getpass import getpass

## 2. LLM Provider Configuration

Choose one of the following sections to configure your preferred LLM provider. You only need to run one of these configuration blocks.

### Option A: OpenAI Configuration

In [ ]:
# Check if OpenAI API key is already set in environment
if 'OPENAI_API_KEY' not in os.environ:
    # Prompt user to enter API key securely
    api_key = getpass("Enter your OpenAI API key: ")
    os.environ['OPENAI_API_KEY'] = api_key
    print("OpenAI API key has been set.")
else:
    print("OpenAI API key is already set in environment.")

model_id = "openai:gpt-5"

### Option B: Google AI Configuration

In [16]:
# Check if Google AI API key is already set in environment
if 'GOOGLE_API_KEY' not in os.environ:
    # Prompt user to enter API key securely
    api_key = getpass("Enter your Google AI API key: ")
    os.environ['GOOGLE_API_KEY'] = api_key
    print("Google AI API key has been set.")
else:
    print("Google AI API key is already set in environment.")

model_id = "google_genai:gemini-2.5-flash"

Google AI API key is already set in environment.


### Option C: Azure OpenAI Configuration

In [ ]:
if "AZURE_OPENAI_API_KEY" not in os.environ:
    os.environ["AZURE_OPENAI_API_KEY"] = getpass("Enter your Azure API key: ")
    print("Azure OpenAI API key has been set.")
else:
    print("Azure OpenAI API key is already set in environment.") 

if "AZURE_OPENAI_ENDPOINT" not in os.environ:
    os.environ["AZURE_OPENAI_ENDPOINT"] = input("Enter your Azure OpenAI endpoint: ")
    print("Azure OpenAI endpoint has been set.")
else:
    print("Azure OpenAI endpoint is already set in environment.")

os.environ["OPENAI_API_VERSION"] = '2024-12-01-preview'  
model_id = "azure_openai:gpt-5-mini"

## 3. Agent System Prompt

Define the system prompt that will guide the agent's behavior when interacting with D365FO.

In [3]:
SYSTEM_PROMPT = """
You are a D365FO document download assistant. Your task is to help users find assistance using the Microsoft Dynamics 365 Finance and Operations (D365FO) tools and resources.
"""

## 4. D365FO Authentication Setup

Configure the credentials needed to connect to your D365FO environment. These credentials are used to authenticate with the D365FO APIs.

In [4]:
D365FO_CLIENT_ID = os.getenv("D365FO_CLIENT_ID") or input("Enter D365FO Client ID: ")
D365FO_CLIENT_SECRET = os.getenv("D365FO_CLIENT_SECRET") or getpass("Enter D365FO Client Secret: ")
D365FO_TENANT_ID = os.getenv("D365FO_TENANT_ID") or input("Enter D365FO Tenant ID: ")

## 5. MCP Client Setup

Initialize the Multi-Server MCP Client to connect to the D365FO MCP server running in a Docker container. This client provides access to all D365FO operations through MCP tools.

In [5]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {
        "d365fo": {
            "command": "docker",
            # Replace with absolute path to your math_server.py file
            "args": [
                "run",
				"--rm",
                "-i",
                "-e",
                f"D365FO_CLIENT_ID={D365FO_CLIENT_ID}",
                "-e",
                f"D365FO_CLIENT_SECRET={D365FO_CLIENT_SECRET}",
                "-e",
                f"D365FO_TENANT_ID={D365FO_TENANT_ID}",
                "-v",
                "d365fo-mcp:/home/mcp_user/",
                "ghcr.io/mafzaal/d365fo-client:latest"
			],
            "transport": "stdio",
        }
      
    }
)
tools = await client.get_tools()
print(f"Number of totals: {len(tools)}")

Number of totals: 43


## 6. Basic D365FO Connection Testing

Before building the agent, let's test the basic connection to D365FO and explore some available tools.

### 6.1 Test Connection Tool

In [6]:
for tool in tools:
    if tool.name == "d365fo_test_connection":
        d365fo_test_connection = tool
        break
print(d365fo_test_connection) # type: ignore

result = await d365fo_test_connection.ainvoke({"profile": "onebox"})  # type: ignore
print(result)

name='d365fo_test_connection' description='Test connection to D365FO environment.\n\nArgs:\n    profile: Optional profile name to test (uses default if not specified)\n\nReturns:\n    JSON string with connection test results\n' args_schema={'properties': {'profile': {'default': 'default', 'title': 'Profile', 'type': 'string'}}, 'title': 'd365fo_test_connectionArguments', 'type': 'object'} response_format='content_and_artifact' coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x725c3ae20900>
{
  "status": true
}


### 6.2 Call Action Tool for Document Generation

Locate and test the action calling tool that will be used for generating documents from D365FO reports.

In [7]:
for tool in tools:
    if "call_action" in tool.name.lower():
        d365fo_call_action_tool = tool
        break
print(f"Call action tool: {d365fo_call_action_tool.name}") # type: ignore

Call action tool: d365fo_call_action


In [8]:
tool_payload = {
    "action_name": "RunCopilotReport",
    "entity_name": "SrsFinanceCopilots",
    "parameters": {
        "_contractName": "SrsCopilotArgsContract",
        "_controllerArgsJson":"""{
            "DataTableName": "CustInvoiceJour",
            "DataTableFieldName": "InvoiceId",
            "DataTableFieldValue": "CIV-000205"
        }""",
        "_controllerName": "SalesInvoiceController",
        "_legalEntityName": "USMF",
        "_reportParameterJson": "{}"
    }
}
response = await d365fo_call_action_tool.ainvoke(tool_payload)
print(response)

{
  "actionName": "RunCopilotReport",
  "success": true,
  "result": {
    "@odata.context": "https://usnconeboxax1aos.cloud.onebox.dynamics.com/data/$metadata#Edm.String",
    "value": "..."
  }
}


### 6.3 Direct Document Generation Tool

Custom tool that use call action direct invocation of the RunCopilotReport action to generate a PDF document for a specific invoice.

In [9]:
from datetime import datetime
from pathlib import Path
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
import base64
import os
from typing import Dict, Any
import json

class PDFDownloadInput(BaseModel):
    document_id: str = Field(description="The ID of the document to save")
    legal_entity: str = Field(description="The legal entity of the document")
    controller_name: str = Field(default="SalesInvoiceController",description="The controller name to use")
    data_table: str = Field(default="CustInvoiceJour", description="The data table name")
    data_field: str = Field(default="InvoiceId",description="The data field name")
    document_type: str = Field(default="SalesInvoice",description="The type of document to save")
    reports_directory: str = Field(default="./Reports",description="The directory where reports are stored")
    profile: str = Field(default="default",description="The profile to use for D365FO connection")

async def download_srs_document(   
        document_id: str,
        legal_entity: str,
        controller_name: str = "SalesInvoiceController",
        data_table: str = "CustInvoiceJour",
        data_field: str = "InvoiceId",
        document_type: str = "SalesInvoice",
        reports_directory: str = "./Reports",   
        profile: str = "default"           ) -> str:
    """
    Convert a base64 string to binary and save as a PDF file.
    
    Args:
        document_id (str): The ID of the document to save
        legal_entity (str): The legal entity of the document
        controller_name (str): The controller name to use. Example: SalesInvoiceController, FreeTextInvoiceController, CustDebitCreditNoteController, SalesConfirmController, PurchPurchaseOrderController
        data_table (str): The data table name. Example: CustInvoiceJour, CustConfirmJour, VendPurchOrderJour
        data_field (str): The data field name. Example: InvoiceId, ConfirmId, SalesId, PurchId
        document_type (str): The type of document to save. Example: Sales Invoice, Free Text Invoice, Debit/Credit Note, Sales Confirmation, Purchase Order
        reports_directory (str): The directory where to save the file
        profile (str): The profile to use for D365FO connection
    
    Returns:
        Path to the saved PDF file
    Raises:
        Exception: If any error occurs during the process
    """

    save_directory = Path(reports_directory)                                                                               
    # Ensure save directory exists                                                                                   
    save_directory.mkdir(parents=True, exist_ok=True)                                                                
                                                                                                                

    # Basic validation 
    if not legal_entity or not document_id:                                                                           
        raise Exception(f"Invalid input for document_id={document_id}, legal_entity={legal_entity}. Skipping.")                      
                                                                                                                   
    # Build controller args JSON string                                                                          
    controller_args = {                                                                                          
        "DataTableName": data_table,                                                                             
        "DataTableFieldName": data_field,                                                                        
        "DataTableFieldValue": document_id                                                                            
    }                                                                                                            
    controller_args_json = json.dumps(controller_args) 
    print(controller_args_json)                                                          
                                                                                                                
    parameters = {                                                                                               
        "_contractName": "SrsCopilotArgsContract",                                                               
        "_controllerArgsJson": controller_args_json,                                                             
        "_controllerName": controller_name,                                                                      
        "_legalEntityName": legal_entity,                                                                        
        "_reportParameterJson": "{}"                                                                             
    }                                                                                                            
                                                                                                                

    action_arguments = {
        "action_name":"RunCopilotReport",    
        "parameters":parameters,                                                
        "entity_name":"SrsFinanceCopilots",                                                    
        "profile":profile
    } 

    response_raw = await d365fo_call_action_tool.ainvoke(action_arguments)

    response = json.loads(response_raw)
                                                     
                                                                                                                
    # Try to extract base64 from expected location response["result"]["value"]                                   
    base64_data = None                                                                                           
    if isinstance(response, dict):                                                                               
        result = response.get("result") if response.get("result") is not None else response.get("Result")        
        if isinstance(result, dict):                                                                             
            base64_data = result.get("value") or result.get("Value")                                             
        # fallback: maybe direct value                                                                           
        if base64_data is None:                                                                                  
            base64_data = response.get("value") or response.get("Value")                                         
                                                                                                                
    if not base64_data:                                                                                          
        raise Exception(f"ERROR: No base64 PDF content found in response for {document_id}. Skipping.")                         
                                                                                                         
                                                                                                                
    try:                                                                                                         
        pdf_bytes = base64.b64decode(base64_data)                                                                
    except Exception as e:                                                                                       
        raise Exception(f"ERROR: Failed to decode base64 for {document_id}: {e}")                                               
                                                                                                         
                                                                                                                
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")                                                         
    filename = f"{document_type}_{document_id}_{legal_entity}_{timestamp}.pdf"                                             
    save_path = save_directory / filename                                                                        
                                                                                                                
    try:                                                                                                         
        save_path.write_bytes(pdf_bytes)                                                                         
        full_path = str(save_path.resolve())                                                                     
        return full_path                                                                            
    except Exception as e:                                                                                       
        raise Exception(f"ERROR: Failed to write PDF for {document_id} to {save_path}: {e}")  
    


tool_description = """
Download SRS document from D365FO and save as PDF.
Use the following table to select appropriate controller, table, and field names based on user request:

| Controller | Table | Field Name | Field Type | Document Type |
|-----------|-------|-----------|------------|---------------|
| `SalesInvoiceController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Sales Invoice |
| `FreeTextInvoiceController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Free Text Invoice |
| `CustDebitCreditNoteController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Debit/Credit Note |
| `SalesConfirmController` | `CustConfirmJour` | `ConfirmId` or `SalesId` | Confirm/Sales ID | Sales Confirmation |
| `PurchPurchaseOrderController` | `VendPurchOrderJour` | `PurchId` | Purchase Order ID | Purchase Order |

Also, user can provide custom controller, table, and field names as needed in query.
"""
# Create the LangGraph tool
download_srs_document_tool = StructuredTool.from_function(
    name="download_srs_document",
    description=tool_description,
    coroutine=download_srs_document,
    args_schema=PDFDownloadInput
)

print("PDF save tool created successfully!")
print(f"Tool name: {download_srs_document_tool.name}")
print(f"Tool description: {download_srs_document_tool.description}")

PDF save tool created successfully!
Tool name: download_srs_document
Tool description: Download SRS document from D365FO and save as PDF.
Use the following table to select appropriate controller, table, and field names based on user request:

| Controller | Table | Field Name | Field Type | Document Type |
|-----------|-------|-----------|------------|---------------|
| `SalesInvoiceController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Sales Invoice |
| `FreeTextInvoiceController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Free Text Invoice |
| `CustDebitCreditNoteController` | `CustInvoiceJour` | `InvoiceId` | Invoice ID | Debit/Credit Note |
| `SalesConfirmController` | `CustConfirmJour` | `ConfirmId` or `SalesId` | Confirm/Sales ID | Sales Confirmation |
| `PurchPurchaseOrderController` | `VendPurchOrderJour` | `PurchId` | Purchase Order ID | Purchase Order |

Also, user can provide custom controller, table, and field names as needed in query.


## 7. Custom Document Download Tool

Create a sophisticated tool that can download various types of D365FO documents as PDF files. This tool handles:
- Different document types (Sales Invoices, Purchase Orders, etc.)
- Base64 to PDF conversion
- File naming and organization
- Error handling and validation

The tool supports multiple document types with the following mapping:

| Document Type | Controller | Table | Field |
|---------------|------------|-------|-------|
| Sales Invoice | SalesInvoiceController | CustInvoiceJour | InvoiceId |
| Free Text Invoice | FreeTextInvoiceController | CustInvoiceJour | InvoiceId |
| Debit/Credit Note | CustDebitCreditNoteController | CustInvoiceJour | InvoiceId |
| Sales Confirmation | SalesConfirmController | CustConfirmJour | ConfirmId/SalesId |
| Purchase Order | PurchPurchaseOrderController | VendPurchOrderJour | PurchId |

### 7.1 Test the Custom Download Tool

Test the custom document download tool with a specific invoice to ensure it works correctly.

In [10]:
report_file_path = await download_srs_document_tool.ainvoke(
    {
        "document_id": "CIV-000205",
        "legal_entity": "USMF",
        "controller_name": "SalesInvoiceController",
        "data_table": "CustInvoiceJour",
        "data_field": "InvoiceId",
        "document_type": "SalesInvoice",
        "reports_directory": "./Reports",   
        "profile": "default"
    })

print(f"Report saved at: {report_file_path}")

{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "CIV-000205"}
Report saved at: /home/mafzaal/sources/d365fo-mcp-prompts/langgraph-mcp/Reports/SalesInvoice_CIV-000205_USMF_20251018_230625.pdf


## 8. Create the LangGraph Agent

Now we'll create the intelligent agent that combines all the D365FO tools with our custom document download tool. The agent can understand natural language requests and execute the appropriate D365FO operations.

### 8.1 Agent Initialization and Capabilities Test

In [20]:
from langchain.agents import create_agent
agent = create_agent(model_id, tools + [download_srs_document_tool],system_prompt=SYSTEM_PROMPT)
agent_response = await agent.ainvoke({"messages": "What can the d365fo agent do for me?"}) # type: ignore
# print all AI and tool call message in the response
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

================================== Ai Message ==================================

I can help you with a wide range of tasks related to Microsoft Dynamics 365 Finance and Operations (D365FO). Here's a summary of my capabilities:

**Metadata and Schema Exploration:**
*   **Get database schema:** I can provide comprehensive schema information for the D365FO metadata database, including tables, columns, indexes, and relationships.
*   **Get table information:** I can give you detailed information about specific database tables, including column definitions, keys, indexes, and even sample data.
*   **Execute SQL queries:** I can execute `SELECT` queries against the D365FO metadata database to help you analyze metadata patterns, generate reports, and gain insights into the D365FO structure.
*   **Search entities:** I can search for D365FO data entities based on keywords, category, and whether they are enabled for data management or OData services.
*   **Get entity schema:** I can provide the

### 8.2 Test Profile Management

In [21]:
agent_response = await agent.ainvoke({"messages": "List F&O profiles"}) # type: ignore
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  d365fo_list_profiles (41ac5397-eff5-4c6f-b51b-7d386a87adde)
 Call ID: 41ac5397-eff5-4c6f-b51b-7d386a87adde
  Args:
================================= Tool Message =================================
Name: d365fo_list_profiles

{
  "totalProfiles": 1,
  "profiles": [
    {
      "base_url": "https://usnconeboxax1aos.cloud.onebox.dynamics.com",
      "verify_ssl": false,
      "timeout": 60,
      "credential_source": {
        "source_type": "environment",
        "client_id_var": "D365FO_CLIENT_ID",
        "client_secret_var": "D365FO_CLIENT_SECRET",
        "tenant_id_var": "D365FO_TENANT_ID"
      },
      "metadata_cache_dir": "/home/mcp_user/.cache/d365fo-client",
      "enable_metadata_cache": true,
      "use_cache_first": true,
      "cache_ttl_seconds": 300,
      "max_memory_cache_size": 1000,
      "enable_fts_search": true,
      "use_label_cache": true,
      "label_cache_expiry_min

### 8.3 Test Connection Status

In [22]:
agent_response = await agent.ainvoke({"messages": "test connection with F&O profile"}) # type: ignore
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  d365fo_test_connection (e5e1b76e-565d-413d-90a3-b7b6b6c08e1b)
 Call ID: e5e1b76e-565d-413d-90a3-b7b6b6c08e1b
  Args:
================================= Tool Message =================================
Name: d365fo_test_connection

{
  "status": true
}
================================== Ai Message ==================================

Connection to D365FO is successful.


### 8.4 Get Environment Information

In [23]:
agent_response = await agent.ainvoke({"messages": "get env info"}) # type: ignore
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  d365fo_get_environment_info (26b73924-0068-4727-b18b-3c05465faa96)
 Call ID: 26b73924-0068-4727-b18b-3c05465faa96
  Args:
================================= Tool Message =================================
Name: d365fo_get_environment_info

{
  "base_url": "https://usnconeboxax1aos.cloud.onebox.dynamics.com",
  "versions": {
    "application": "10.0.43",
    "platform": "7.0.7521.60",
    "build": "10.0.2177.37"
  },
  "connectivity": true,
  "metadata_info": {
    "cache_directory": "/home/mcp_user/.cache/d365fo-client",
    "cache_version": "2.0",
    "statistics": {
      "data_entities_count": 5268,
      "public_entities_count": 3993,
      "entity_properties_count": 70102,
      "navigation_properties_count": 6972,
      "entity_actions_count": 354,
      "enumerations_count": 2118,
      "labels_cache_count": 293,
      "environment_statistics": {
        "total_environments": 1,
        

## 9. Advanced Agent Use Cases

Now let's test more complex scenarios that demonstrate the agent's ability to understand natural language requests and perform sophisticated D365FO operations.

### 9.1 Single Document Download

Test downloading a specific customer invoice by ID.

In [24]:
agent_response = await agent.ainvoke({"messages": "Download customer invoice CIV-000708 for legal entity USMF"}) # type: ignore
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "CIV-000708"}
================================== Ai Message ==================================
Tool Calls:
  download_srs_document (3094915d-26a8-499f-afae-3a6924db4231)
 Call ID: 3094915d-26a8-499f-afae-3a6924db4231
  Args:
    legal_entity: USMF
    data_table: CustInvoiceJour
    controller_name: SalesInvoiceController
    document_type: Sales Invoice
    document_id: CIV-000708
    data_field: InvoiceId
================================= Tool Message =================================
Name: download_srs_document

/home/mafzaal/sources/d365fo-mcp-prompts/langgraph-mcp/Reports/Sales Invoice_CIV-000708_USMF_20251018_231300.pdf
================================== Ai Message ==================================

The customer invoice CIV-000708 for legal entity USMF has been downloaded to /home/mafzaal/sources/d365fo-mcp-prompts/langgraph-mcp/Reports/Sales Invoice_CIV-000708_USMF_20251018_231300.pdf

### 9.2 Complex Multi-Step Operation

Test a complex scenario where the agent needs to:
1. Query D365FO to get recent invoices
2. Download PDF documents for those invoices

This demonstrates the agent's ability to chain multiple operations together based on a single natural language request.

In [28]:
agent_response = await agent.ainvoke({"messages": "Get 5 recent invoices for legal entity USMF using CustInvoiceJourBiEntities data entity and then download invoices. Get data entity schema using `CustInvoiceJourBiEntity` entity name  to build query parameters."}) # type: ignore
for message in agent_response['messages']:
   if message.type in ['tool', 'ai']:
        message.pretty_print()

{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "FTI-00000021"}
{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "FTI-00000022"}
{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "FTI-00000014"}
{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "FTI-00000015"}
{"DataTableName": "CustInvoiceJour", "DataTableFieldName": "InvoiceId", "DataTableFieldValue": "FTI-00000006"}
================================== Ai Message ==================================
Tool Calls:
  d365fo_get_entity_schema (53cd26af-4da8-417a-8e9b-9a91245db738)
 Call ID: 53cd26af-4da8-417a-8e9b-9a91245db738
  Args:
    entityName: CustInvoiceJourBiEntity
    include_properties: True
================================= Tool Message =================================
Name: d365fo_get_entity_schema

{
  "name": "CustInvoiceJourBiEntity",
  "entity_

## 10. Summary and Next Steps

This notebook demonstrates a complete D365FO MCP LangGraph agent that can:

- **Connect to D365FO**: Authenticate and establish connections to D365FO environments
- **Query Data**: Retrieve information from various D365FO data entities
- **Download Documents**: Generate and download PDF documents for invoices, purchase orders, and other document types
- **Natural Language Interface**: Understand complex requests and execute multi-step operations
- **Error Handling**: Gracefully handle authentication, network, and data issues

### Key Features Demonstrated:

1. **Multi-Provider LLM Support**: Works with OpenAI, Google AI, and Azure OpenAI
2. **MCP Integration**: Seamless connection to D365FO through Model Context Protocol
3. **Custom Tool Development**: Extensible architecture for adding new D365FO operations
4. **Document Management**: Automated PDF generation and file organization
5. **Agent Intelligence**: Context-aware responses and multi-step task execution

### Potential Extensions:

- Add support for more document types (Picking and packing lists, Statements, etc.)
- Implement batch operations for processing multiple documents
- Add data analysis capabilities for financial reporting
- Integrate with other ERP modules beyond Finance
- Create workflow automation for common business processes

### Troubleshooting Tips:

- Ensure Docker is running and the D365FO MCP server container is accessible
- Verify D365FO credentials and environment connectivity
- Check API rate limits and adjust retry logic as needed
- Monitor log output for detailed error information